In [1]:
pip install torch transformers huggingface_hub requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Fetch HF_TOKEN
hf_token = os.environ.get("HF_TOKEN")

In [4]:
import os
import requests
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login

Authenticate with Hugging Face (Gemma)

In [ ]:
login(token=hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


: 

# Loading a Hugging Face Model : Gemma Model

In [ ]:
#model_name = "google/gemma-2b-it" #This is a big model and codespace is not able to run it, replacing it with a smaller one
model_name = "microsoft/phi-2"  # A smaller model for demonstration

tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
# Downloads and loads the tokenizer for DistilGPT2. 
# The tokenizer converts raw text into tokens (numbers) that the model can understand. 
# It uses the same byte-level BPE as GPT-2. This is where the text is converted to embeddings (indirectly): the tokenizer outputs token IDs,
# which the model then turns into embeddings in its first layer.

# Downloads and loads the pre-trained DistilGPT2 model. This is the neural network that generates text, given the input tokens.
model = AutoModelForCausalLM.from_pretrained(model_name, token=hf_token)

#The pipelines are a great and easy way to use models for inference. These pipelines are objects that abstract most of the complex code from the library, offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering.
# Pipeline we get from Hugging Face, text-generation parameter should be compatible with the model we are using.
#Wraps the model and tokenizer in a high-level pipeline for easy text generation. Now you can just call llm("Your prompt") to generate text. max_new_tokens=256 limits the output length.
llm = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Weather tooling function : Our function to get the weather. Ideally our LLM Model should be able to call this function according to the response.

In [ ]:
# --- 3. Weather Tool Function (using Open-Meteo, free, no auth needed) ---

def get_weather(city):
    # Geocode city to lat/lon (using Open-Meteo's geocoding)
    geo = requests.get(f"https://geocoding-api.open-meteo.com/v1/search?name={city}").json()
    '''
    - Defines a function `get_weather` that takes a city name as input.
    - Sends a GET request to the Open-Meteo geocoding API to find the latitude and longitude of the city. The response is converted to JSON.
    '''

    if not geo.get("results"):
        return f"Could not find location for {city}."
    '''- Checks if the API response contains a `"results"` key. If not, it returns a message saying the location couldn't be found.'''

    lat, lon = geo["results"][0]["latitude"], geo["results"][0]["longitude"]

    # Get weather, Sends another GET request to the Open-Meteo weather API, using the latitude and longitude found earlier, to get the current weather. The response is converted to JSON.
    weather = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    ).json()
    if "current_weather" in weather:
        temp = weather["current_weather"]["temperature"]
        desc = f"The current temperature in {city} is {temp}°C."
        return desc
    else:
        return "Weather data not available."
    

In [ ]:
print(f"Weather Tool Ready. Example usage: {get_weather('Bangalore')}") # Testing the weather function: Weather Tool Ready. Example usage: The current temperature in Bangalore is 27.0°C.

Weather Tool Ready. Example usage: The current temperature in Bangalore is 27.5°C.


# React Prompting : Reasong and Acting :

Here’s a clear explanation of **ReAct prompting**—first in layman’s terms, then in technical detail, with a practical example showing how prompts and model responses work at each step.

## **Layman’s Explanation**

**ReAct prompting** stands for **Reasoning and Acting**.  
It’s a way to get AI to not just “think” (reason) about a problem, but also “do” (act) something step by step—just like a person solving a puzzle:

- **Reason:** The AI explains what it’s thinking or planning.
- **Act:** The AI takes an action (like looking something up, using a tool, or searching a database).
- **Observe:** The AI sees what happened after the action.
- **Repeat:** The AI reasons again, takes another action if needed, and continues until it finds the answer.

**Why use it?**  
It helps the AI solve complex problems more accurately, transparently, and flexibly—especially when it needs to interact with tools or external information[1][2][3][4][5].

## **Technical Deep Dive**

### **What is ReAct Prompting?**

- **ReAct** is a prompting technique that combines:
  - **Reasoning:** The AI breaks down the problem, thinks through possible solutions, and plans what to do next.
  - **Acting:** The AI takes concrete steps (actions) based on its reasoning, such as searching, calculating, or calling a function.
- This process is **iterative**: The AI alternates between reasoning and acting, using the outcome of each action to inform its next step[1][2][3][4][5].

### **Key Benefits**

- **Transparency:** You can see the AI’s thought process.
- **Accuracy:** The AI checks its own work at each step.
- **Adaptability:** The AI can adjust its approach if new information appears.
- **Reduces Hallucination:** By grounding actions in real observations, the AI is less likely to make things up[1][4][5].

### **How ReAct Prompting Works (Step-by-Step Example)**

**Scenario:**  
Suppose you want the AI to answer:  
*"Who is the current president of the United States, and what is their age?"*

**Prompt Structure:**  
You instruct the AI to alternate between Thought (reason), Action (do), and Observation (see result).

#### **Step-by-Step Example**

**User Prompt:**  
```
Answer the question step by step. For each step, alternate between:
- Thought: What should you do next?
- Action: Take an action (e.g., search, calculate).
- Observation: What did you find?
Repeat until you can answer the question.
Question: Who is the current president of the United States, and what is their age?
```

**Model Response:**

1. **Thought:** I need to find out who the current president of the United States is.
2. **Action:** Search["current president of the United States"]
3. **Observation:** The search result says "Joe Biden is the current president."
4. **Thought:** Now I need to find out Joe Biden's age.
5. **Action:** Search["Joe Biden age"]
6. **Observation:** The search result says "Joe Biden is 82 years old."
7. **Final Answer:** The current president of the United States is Joe Biden, who is 82 years old.

### **How Is This Used Further?**

- **In AI agents:** Each action could be a real API call, database query, or tool use.
- **In customer support:** The AI could reason about a complaint, look up order status, and recommend solutions.
- **In research:** The AI could plan a series of searches, summarize findings, and synthesize an answer.

## **Summary Table**

| Step       | What Happens                                    | Example in Practice                                 |
|------------|-------------------------------------------------|-----------------------------------------------------|
| Thought    | AI decides what to do next                      | "I need to find the president's name."              |
| Action     | AI performs an action (search, tool, etc.)      | Search["current president of the US"]               |
| Observation| AI sees the result of the action                | "Result: Joe Biden is the president."               |
| Repeat     | AI reasons again, takes next action if needed   | "Now, find Joe Biden's age."                        |
| Final Answer| AI gives the complete answer                   | "Joe Biden, age 82."                                |

In [ ]:
# --- 4. ReAct Loop Implementation ---
def react_loop(user_question):
    history = []  # To store the AI's reasoning and actions
    observation = ""
    max_steps = 5  # Prevent infinite loops
    for step in range(max_steps):
        # 1. Build prompt with history, user question, and last observation
        prompt = (
            "You are an assistant that can reason and use tools.\n"
            "When you need to use a tool, write: Action: get_weather(city)\n"
            "I will execute the action and return the result as Observation.\n"
            "Repeat Thought/Action/Observation until you can answer.\n"
            f"User question: {user_question}\n"
        )
        for entry in history:
            prompt += entry + "\n"
        if observation:
            prompt += f"Observation: {observation}\n"

        prompt += "Thought:"

        # 2. Get AI's next step
        response = llm(prompt, do_sample=False)[0]['generated_text']
        # Extract only the new completion after the last prompt
        next_line = response[len(prompt):].strip().split("\n")[0]

        print(f"AI: {next_line}")

        # 3. Check if AI wants to act
        if "Action:" in next_line:
            # Parse city from action
            import re
            match = re.search(r'get_weather\((.*?)\)', next_line)
            if match:
                city = match.group(1).strip().strip('"').strip("'")
                observation = get_weather(city)
                history.append(f"Thought:{next_line}")
                history.append(f"Action: get_weather({city})")
                history.append(f"Observation: {observation}")
            else:
                observation = "Invalid action format."
                history.append(f"Thought:{next_line}")
                history.append("Action: none")
                history.append(f"Observation: {observation}")
        elif "Final Answer:" in next_line or "Answer:" in next_line:
            print(next_line)
            break
        else:
            # If just reasoning, add to history
            history.append(f"Thought:{next_line}")


In [ ]:
user_q = "What is the weather in Bangalore?"  # <-- Change this to any city/question you want
react_loop(user_q)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI: What is the weather in Bangalore?


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI: What is the weather in Bangalore?


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI: What is the weather in Bangalore?


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI: What is the weather in Bangalore?
AI: What is the weather in Bangalore?
